In [30]:
import requests
import os
import json
from google.cloud import storage, bigquery
from dataverk_vault import api as vault_api

In [31]:
vault_api.set_secrets_as_envs()

In [32]:
with open("/home/jovyan/creds.json", "w") as f:
    f.write(os.environ["STYRK_BUCKET_CREDS"])
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/creds.json"

In [33]:
client = storage.Client()

In [34]:
bucket = client.get_bucket("styrk-bucket")

In [35]:
blob = bucket.blob("styrk-koder/styrk.json")

In [36]:
data = json.loads(blob.download_as_string().decode())

In [37]:
columns = list(data["classificationItems"][0].keys())

In [38]:
create_or_replace_table = "CREATE OR REPLACE TABLE styrk.koder ( "
for column in columns:
    create_or_replace_table += column + " STRING, "
create_or_replace_table = create_or_replace_table[:-2]
create_or_replace_table += " );"

print(create_or_replace_table)

CREATE OR REPLACE TABLE styrk.koder ( code STRING, parentCode STRING, level STRING, name STRING, shortName STRING, notes STRING );


In [39]:
bq_client = bigquery.Client()

In [40]:
query = bq_client.query(create_or_replace_table)
query.result()

In [41]:
query = f"INSERT styrk.koder ("
for column in columns:
    query += column + ", "
query = query[:-2]
query += """) 
VALUES\n"""

for code in data["classificationItems"]:
    query += "("
    for key in code.keys():
        query += f""""{code.get(key).replace('"', '').replace(',', '')}", """
    query = query[:-2]
    query += "), \n"
query = query[:-3]

In [42]:
query = bq_client.query(query)
query.result()